# Numerical Grids in NRPy+

### NRPy+ Source Code for this module: [grid.py](../edit/grid.py)

Solving partial differential equations on the computer with finite difference techniques requires that we sample our solutions to these equations on numerical grids. We call all sampled functions that are stored on our numerical grid *gridfunctions*. NRPy+'s grid module adds the capability of registering gridfunctions in NRPy, setting basic parameters of a numerical grid, and providing functions to other modules regarding reading and writing of gridfunctions to memory in the C code.

Parameters in this module include:
* **grid::DIM** -- the dimension of the grid (e.g., a 3D numerical grid will have grid::DIM=3)
* **grid::Nx\[DIM\]** -- an integer array yielding the size of the grid in each direction. E.g., in Cartesian coordinates Nx\[0\] will be set to the number of grid points in the x direction. *This is a parameter that is set at C runtime, not in NRPy+; NRPy+ simply generates the declaration of this parameter in the C code.*
* **grid::MemAllocStyle** -- how the gridfunction is allocated in memory in the C code. This is used when generating the C code to ensure that gridfunctions are read as sequentially in memory as possible, to avoid [cache misses](https://en.wikipedia.org/wiki/CPU_cache#CACHE-MISS). There are currently two MemAllocStyles supported:
    * If the following loop accesses the grid function in a contiguous manner in memory, we set "grid::MemAllocStyle=012":
        * for(int i0=0;i0<Nx\[0\];i0++) for(int i1=0;i1<Nx\[1\];i1++) for(int i2=0;i2<Nx\[2\];i2++) {
    * Alternatively, the "grid::MemAllocStyle=210" is the reverse:
        * for(int i2=0;i2<Nx\[2\];i2++) for(int i1=0;i1<Nx\[1\];i1++) for(int i0=0;i0<Nx\[0\];i0++) {
* **grid::GridFuncMemAccess** -- specifies how gridfunction data is accessed from memory. For example,
    * In the Einstein Toolkit (grid::GridFuncMemAccess="ETK"), the datum from gridfunction dummy at point (i0,j0,k0) is accessed via "dummyGF\[CCTK_GFINDEX3D(cctkGH,i0,j0,k0)\]".
    * In SENR (grid::GridFuncMemAccess="SENRlike"), the datum from gridfunction dummy in gridfunction array gfarry at point (i0,j0,k0) is accessed via "gfarray\[IDX4D(DUMMYGF,i0,j0,k0)\]".
    * *Special note*: NRPy+ is code agnostic; additional types can be easily be added by modifying the function gfaccess() in the NRPy+ grid module. It should only require about 5 lines of code.

Functions in this module include:
* **gfaccess(gfarrayname = "",varname = "",ijklstring = "")**: Given a gridfunction array name, a variable name, and a string indicating the coordinates of the point in memory, return the string to access the gridfunction in memory at this data point. See grid::GridFuncMemAccess parameter above for a complete description.
* **register_gridfunctions(gf_type,gf_names)**: returns either a single gridfunction of list of gridfunctions as SymPy variables
    * gf_type can be set to either "EVOL" or "AUX".
        * Setting to "EVOL" denotes a grid function that will need to be stepped forward in time within the C code's timestepping algorithm.
        * Setting to "AUX" denotes any other grid function.
    * gf_names can be a single gridfunction or a set of gridfunctions, all with the same gf_type.
* **variable_type(var)**: First searches the list of registered gridfunctions and parameters for gridfunctions or C parameters; then outputs "gridfunction" or "Cparameter" if one or the other is found, respectively. Otherwise it will output "other"

Let's now register a gridfunction called "phi" in the "in_gfs" gridfunction array, then print C code that specifies how to access the gridfunction from the single point in memory (i0,i1,i2). Next we will demonstrate that phi is a normal SymPy variable, and finally we will confirm that its type is "gridfunction" using variable_type():

In [1]:
import grid as gri

# Register gridfunction phi, as type "AUX". 
# WARNING: register_gridfunctions can only be run once on a given gridfunction;
#          you'll need to reset the Jupyter kernel before trying again:
phi = gri.register_gridfunctions("AUX","phi")

print("Here's how to access the gridfunction phi, in grid array in_gfs, at point (i0,i1,i2):")
print(gri.gfaccess("in_gfs","phi","i0,i1,i2"))

# Note that phi can now be used as a usual SymPy variable:
print("To demonstrate that phi is now just a regular SymPy variable, we will now print its square:")
print(phi**2)

print("\"phi\" is of type \""+ gri.variable_type(phi)+"\"")

Here's how to access the gridfunction phi, in grid array in_gfs, at point (i0,i1,i2):
in_gfs[IDX4(PHIGF, i0,i1,i2)]
To demonstrate that phi is now just a regular SymPy variable, we will now print its square:
phi**2
"phi" is of type "gridfunction"
